In [1]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

24/03/20 13:48:26 WARN Utils: Your hostname, Onus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.169 instead (on interface en0)
24/03/20 13:48:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/onuh/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/onuh/.ivy2/cache
The jars for the packages stored in: /Users/onuh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-468a41ea-2032-469e-a004-688ce5b342bf;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spar

24/03/20 13:53:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [2]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [3]:
def peek(mini_batch, batch_id):
    first_row = mini_batch.take(1)

    if first_row:
        print(first_row[0])

query = green_stream.writeStream.foreachBatch(peek).start()

24/03/20 13:55:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0m/w6lgq_kd10x33nt6g77_xbj00000gn/T/temporary-2eda90b8-62fd-42fb-8ab7-7e47ec8ff978. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/20 13:55:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/20 13:55:33 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(key=None, value=bytearray(b'{"lpep_pickup_datetime": "2019-10-01 00:26:02", "lpep_dropoff_datetime": "2019-10-01 00:39:58", "PULocationID": 112, "DOLocationID": 196, "passenger_count": 1.0, "trip_distance": 5.88, "tip_amount": 0.0}'), topic='green-trips', partition=0, offset=0, timestamp=datetime.datetime(2024, 3, 20, 13, 0, 23, 405000), timestampType=0)


In [4]:
query.stop()

In [5]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())

In [6]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [ ]:
query = green_stream.writeStream.foreachBatch(peek).start()

24/03/20 14:05:41 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0m/w6lgq_kd10x33nt6g77_xbj00000gn/T/temporary-47496b0f-29c2-46b8-afde-d1abf85c2b21. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/20 14:05:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/20 14:05:41 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Row(lpep_pickup_datetime='2019-10-01 00:26:02', lpep_dropoff_datetime='2019-10-01 00:39:58', PULocationID=112, DOLocationID=196, passenger_count=1.0, trip_distance=5.88, tip_amount=0.0)


In [10]:
query.stop()

In [11]:
# Add a column "timestamp" using the current_timestamp function
green_stream = green_stream.withColumn("timestamp", F.current_timestamp())

green_stream.printSchema()

root
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- timestamp: timestamp (nullable = false)



In [14]:
# Group by: 5 minutes window based on the timestamp column (F.window(col("timestamp"), "5 minutes") "DOLocationID" Order by count
popular_destinations = green_stream \
    .groupBy(F.window(F.col("timestamp"), "5 minutes"), F.col("DOLocationID")) \
    .count() \
    .orderBy(F.desc("count"))

In [15]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

24/03/20 14:33:38 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0m/w6lgq_kd10x33nt6g77_xbj00000gn/T/temporary-e1455943-fd63-4331-a5e1-b1337f34c46b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/03/20 14:33:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/20 14:33:38 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{2024-

-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{2024-

-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{2024-

-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{2024-

...

-------------------------------------------
Batch: 148
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{202

-------------------------------------------
Batch: 149
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{202

-------------------------------------------
Batch: 150
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{202

-------------------------------------------
Batch: 151
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{202

-------------------------------------------
Batch: 152
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|74          |27855|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|42          |24784|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|41          |22042|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|75          |20046|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|129         |18738|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|7           |18130|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|166         |17297|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|236         |12437|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|223         |11970|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|238         |11583|
|{2024-03-20 14:30:00, 2024-03-20 14:35:00}|82          |11308|
|{202

In [ ]:
query.stop()